### Symbolic Regression with PySr


In [1]:
import pandas as pd
import numpy as np
import sys
import os

# Adjust the path to point to external/AlphaPEM
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.sampling.sampler import get_polarisation_curve_samples, build_fixed_parameters, sample_parameters, PARAMETER_RANGES

In [2]:
data = pd.read_pickle('../data/raw/N10_levels4_seed42.pkl')

In [3]:
exploaded_df = data.explode(['ifc', 'Ucell'])
exploaded_df = exploaded_df[0:100]  # Limit to 1000 samples for faster processing

In [4]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np
from pysr import PySRRegressor

vars = list(PARAMETER_RANGES.keys())
vars.append('ifc')

X = exploaded_df[vars].values
y = exploaded_df['Ucell'].values

kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_scores = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = PySRRegressor(
        niterations=100,
        binary_operators=["+", "-", "*", "/"],
        unary_operators=["exp", "log", "sqrt", "square", "cube"],
        loss="loss(x, y) = (x - y)^2",
        model_selection="best",
        maxsize=20,
        maxdepth=5,
        ncyclesperiteration=30,
        verbosity=1,
        output_directory="../models/pysr_equations"
    )
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)

print(f"Average MSE: {np.mean(mse_scores):.4f} ± {np.std(mse_scores):.4f}")


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


c:\Users\dejvi\Documents\GitHub\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\AlphaPEM_env\Lib\site-packages\pysr\sr.py:1036: FutureWarning: `loss` has been renamed to `elementwise_loss` in PySRRegressor. Please use that instead.
  warnings.warn(
c:\Users\dejvi\Documents\GitHub\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\AlphaPEM_env\Lib\site-packages\pysr\sr.py:1036: FutureWarning: `ncyclesperiteration` has been renamed to `ncycles_per_iteration` in PySRRegressor. Please use that instead.
  warnings.warn(
c:\Users\dejvi\Documents\GitHub\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\AlphaPEM_env\Lib\site-packages\pysr\sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
Compiling Julia backend...
[ Info: Started!
[ Info: Final population:
[ Info: Results saved to:
c:\Users\dejvi\Documents\GitHub\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\AlphaPE

───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           9.795e+00  0.000e+00  y = 3.8124
3           3.926e+00  4.571e-01  y = x₁ * 2.6471e-05
4           2.722e+00  3.663e-01  y = square(x₁) * 1.2866e-10
5           8.840e-01  1.125e+00  y = (x₁ * 3.3905e-05) - x₁₇
7           3.228e-01  5.037e-01  y = (x₁ * 3.8752e-05) - (x₁₇ * 1.652)
9           2.938e-01  4.711e-02  y = ((x₁ / 80.941) / x₀) - sqrt(cube(x₁₇))
11          1.925e-01  2.114e-01  y = ((x₁ / 348.88) / (x₀ + -257.11)) - sqrt(cube(x₁₇))
12          1.306e-01  3.882e-01  y = (13.361 - sqrt(cube(x₁₇))) + (square(x₀) / (x₂ / -8.82...
                                      93))
13          1.281e-01  1.906e-02  y = (13.273 - sqrt(cube(x₁₇))) + ((x₀ + -188.85) / (x₂ / -...
                                      6677.6))
15          1.281e-01  7.010e-05  y = (13.263 - sqrt(cube(x₁₇))) + ((x₀ - 191.72) / ((x₁₇ + ...
                  

[ Info: Started!
[ Info: Final population:
[ Info: Results saved to:
c:\Users\dejvi\Documents\GitHub\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\AlphaPEM_env\Lib\site-packages\pysr\sr.py:1036: FutureWarning: `loss` has been renamed to `elementwise_loss` in PySRRegressor. Please use that instead.
  warnings.warn(
c:\Users\dejvi\Documents\GitHub\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\AlphaPEM_env\Lib\site-packages\pysr\sr.py:1036: FutureWarning: `ncyclesperiteration` has been renamed to `ncycles_per_iteration` in PySRRegressor. Please use that instead.
  warnings.warn(
c:\Users\dejvi\Documents\GitHub\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\AlphaPEM_env\Lib\site-packages\pysr\sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           8.798e+00  0.000e+00  y = 3.7584
3           3.745e+00  4.270e-01  y = x₁ * 2.5912e-05
4           3.048e+00  2.060e-01  y = square(x₁) * 1.2521e-10
5           9.143e-01  1.204e+00  y = (x₁ * 3.3801e-05) - x₁₇
7           3.142e-01  5.340e-01  y = (x₂ * 4.3192e-05) - (x₁₇ * 1.858)
9           2.218e-01  1.742e-01  y = ((x₂ * 0.01442) / x₀) - (x₁₇ * 1.826)
11          1.380e-01  2.372e-01  y = (x₂ * ((0.046374 / x₀) - 9.6003e-05)) - (x₁₇ * 1.7337)
13          8.704e-02  2.305e-01  y = (x₂ * ((0.063912 / x₀) - 0.00015313)) - ((x₁₇ * 1.8322...
                                      ) - 0.96466)
15          8.704e-02  2.980e-08  y = (x₂ * ((0.06387 / x₀) - 0.000153)) - (((x₁₇ * 1.1604) ...
                                      - 0.61016) / x₅)
───────────────────────────────────────────────────────────────────────────────────────────────

...

───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           9.056e+00  0.000e+00  y = 3.9015
3           3.676e+00  4.508e-01  y = x₁ * 2.6352e-05
4           2.899e+00  2.374e-01  y = square(x₁ * 1.1163e-05)
5           8.536e-01  1.223e+00  y = (x₁ * 3.4012e-05) - x₁₇
7           3.147e-01  4.989e-01  y = (x₂ * 4.3218e-05) + (x₁₇ * -1.8422)
9           2.317e-01  1.531e-01  y = (x₁₇ * -1.8119) - (x₂ * (-0.014416 / x₀))
11          1.566e-01  1.960e-01  y = (x₁₇ * -1.726) - ((x₂ * 0.0038644) / (243.56 - x₀))
13          9.655e-02  2.418e-01  y = ((x₁₇ * -1.8459) - -1.0552) + ((x₂ * -0.0018965) / (28...
                                      3.63 - x₀))
17          9.148e-02  1.348e-02  y = (x₁ * 7.4866e-06) - ((-1.1022 - (x₁₇ * -1.8247)) + ((x...
                                      ₂ * 0.0010863) / (296.72 - x₀)))
─────────────────────────────────────────────────────────────────────────────

[ Info: Started!
[ Info: Final population:
[ Info: Results saved to:
c:\Users\dejvi\Documents\GitHub\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\AlphaPEM_env\Lib\site-packages\pysr\sr.py:1036: FutureWarning: `loss` has been renamed to `elementwise_loss` in PySRRegressor. Please use that instead.
  warnings.warn(
c:\Users\dejvi\Documents\GitHub\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\AlphaPEM_env\Lib\site-packages\pysr\sr.py:1036: FutureWarning: `ncyclesperiteration` has been renamed to `ncycles_per_iteration` in PySRRegressor. Please use that instead.
  warnings.warn(
c:\Users\dejvi\Documents\GitHub\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\AlphaPEM_env\Lib\site-packages\pysr\sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(



Expressions evaluated per second: 1.560e+05
Progress: 3003 / 3100 total iterations (96.871%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           8.917e+00  0.000e+00  y = 3.7621
3           3.932e+00  4.094e-01  y = x₁ * 2.5886e-05
5           1.065e+00  6.529e-01  y = (x₁ * 3.3687e-05) - x₁₇
7           3.195e-01  6.022e-01  y = (x₁ * 4.0185e-05) - (x₁₇ / 0.54563)
8           3.147e-01  1.492e-02  y = (x₁ * 4.0152e-05) - (x₁₇ / sqrt(x₈))
9           2.579e-01  1.991e-01  y = (x₁ * (0.013372 / x₀)) - (x₁₇ / 0.56216)
11          2.405e-01  3.495e-02  y = ((x₁ * (0.012985 / x₀)) - (x₁₇ * 1.8381)) + x₈
12          2.364e-01  1.738e-02  y = x₈ + (((x₁ * 0.01296) / x₀) - (x₁₇ / sqrt(x₈)))
13          1.135e-01  7.337e-01  y = ((x₁ * 3.3791e-05) - ((x₁₇ + -11.731) * 1.7985)) + (x₀...
      

[ Info: Started!
[ Info: Final population:
[ Info: Results saved to:



Expressions evaluated per second: 2.220e+05
Progress: 2936 / 3100 total iterations (94.710%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           9.444e+00  0.000e+00  y = 3.9314
3           3.667e+00  4.730e-01  y = x₁ * 2.7529e-05
4           3.104e+00  1.666e-01  y = square(x₂) * 1.5726e-10
5           9.125e-01  1.224e+00  y = (x₁ * 3.4499e-05) - x₁₇
7           3.467e-01  4.839e-01  y = (x₂ * 4.3556e-05) - (x₁₇ * 1.8927)
9           2.868e-01  9.487e-02  y = ((x₀ * x₁₇) * -0.0055669) - (x₂ * -4.3479e-05)
11          1.822e-01  2.269e-01  y = (x₂ * 4.2689e-05) - (x₁₇ * ((x₀ * 0.027381) + -7.4798)...
                                      )
12          1.822e-01  5.960e-08  y = (x₂ * 4.2689e-05) - ((log(x₀) + -5.6336) * (x₁₇ * 9.38...
                                      7))
13     